In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 165ms/step - loss: 3.2332 - accuracy: 0.5476 - precision: 0.5196 - recall: 0.4650 - auc: 0.5753
Epoch 2/30
58/58 [==============================] - 10s 173ms/step - loss: 2.2444 - accuracy: 0.6243 - precision: 0.6164 - recall: 0.5257 - auc: 0.6987
Epoch 3/30
58/58 [==============================] - 10s 174ms/step - loss: 1.7673 - accuracy: 0.7317 - precision: 0.7038 - recall: 0.7383 - auc: 0.8013
Epoch 4/30
58/58 [==============================] - 10s 175ms/step - loss: 1.5395 - accuracy: 0.7623 - precision: 0.7554 - recall: 0.7290 - auc: 0.8130
Epoch 5/30
58/58 [==============================] - 10s 177ms/step - loss: 1.3978 - accuracy: 0.7492 - precision: 0.7018 - recall: 0.8084 - auc: 0.8241
Epoch 6/30
58/58 [==============================] - 10s 175ms/step - loss: 1.1751 - accuracy: 0.8313 - precision: 0.8086 - recall: 0.8388 - auc: 0.8842
Epoch 7/30
58/58 [==============================] - 10s 177ms/step - loss: 1.1646 - accu

58/58 [==============================] - 10s 180ms/step - loss: 1.0676 - accuracy: 0.6331 - precision_1: 0.6260 - recall_1: 0.5397 - auc_1: 0.7127
Epoch 24/30
58/58 [==============================] - 10s 180ms/step - loss: 1.0460 - accuracy: 0.6539 - precision_1: 0.6600 - recall_1: 0.5397 - auc_1: 0.7313
Epoch 25/30
58/58 [==============================] - 10s 180ms/step - loss: 1.0265 - accuracy: 0.6681 - precision_1: 0.6457 - recall_1: 0.6472 - auc_1: 0.7355
Epoch 26/30
58/58 [==============================] - 10s 179ms/step - loss: 1.0057 - accuracy: 0.6703 - precision_1: 0.6453 - recall_1: 0.6589 - auc_1: 0.7506
Epoch 27/30
58/58 [==============================] - 10s 180ms/step - loss: 0.9975 - accuracy: 0.6922 - precision_1: 0.6763 - recall_1: 0.6589 - auc_1: 0.7587
Epoch 28/30
58/58 [==============================] - 10s 179ms/step - loss: 0.9670 - accuracy: 0.7087 - precision_1: 0.6709 - recall_1: 0.7430 - auc_1: 0.7865
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 10s 180ms/step - loss: 0.8703 - accuracy: 0.7930 - precision_3: 0.8120 - recall_3: 0.7266 - auc_3: 0.8248
Epoch 15/30
58/58 [==============================] - 10s 181ms/step - loss: 0.8666 - accuracy: 0.8105 - precision_3: 0.7852 - recall_3: 0.8201 - auc_3: 0.8189
Epoch 16/30
58/58 [==============================] - 10s 181ms/step - loss: 0.7508 - accuracy: 0.8434 - precision_3: 0.8701 - recall_3: 0.7827 - auc_3: 0.8535
Epoch 17/30
58/58 [==============================] - 10s 179ms/step - loss: 0.7202 - accuracy: 0.8445 - precision_3: 0.8488 - recall_3: 0.8131 - auc_3: 0.8593
Epoch 18/30
58/58 [==============================] - 10s 180ms/step - loss: 0.6780 - accuracy: 0.8488 - precision_3: 0.8341 - recall_3: 0.8458 - auc_3: 0.8811
Epoch 19/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7662 - accuracy: 0.7930 - precision_3: 0.8504 - recall_3: 0.6776 - auc_3: 0.8070
Epoch 20/30
58/58 [==============================] - 10s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 309ms/step - loss: 4.5248 - accuracy: 0.5761 - precision_5: 0.5527 - recall_5: 0.5023 - auc_5: 0.6181
Epoch 2/30
58/58 [==============================] - 18s 310ms/step - loss: 2.8094 - accuracy: 0.6878 - precision_5: 0.6706 - recall_5: 0.6565 - auc_5: 0.7518
Epoch 3/30
58/58 [==============================] - 18s 309ms/step - loss: 2.1366 - accuracy: 0.7579 - precision_5: 0.7285 - recall_5: 0.7710 - auc_5: 0.8235
Epoch 4/30
58/58 [==============================] - 18s 308ms/step - loss: 1.8070 - accuracy: 0.7667 - precision_5: 0.7529 - recall_5: 0.7477 - auc_5: 0.8456
Epoch 5/30
58/58 [==============================] - 18s 311ms/step - loss: 1.5964 - accuracy: 0.7908 - precision_5: 0.7570 - recall_5: 0.8154 - auc_5: 0.8614
Epoch 6/30
58/58 [==============================] - 18s 314ms/step - loss: 1.4530 - accuracy: 0.7908 - precision_5: 0.7495 - recall_5: 0.8318 - auc_5: 0.8575
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 18s 315ms/step - loss: 0.8240 - accuracy: 0.6605 - precision_6: 0.6093 - recall_6: 0.7687 - auc_6: 0.7251
Epoch 23/30
58/58 [==============================] - 18s 311ms/step - loss: 0.7009 - accuracy: 0.7853 - precision_6: 0.7500 - recall_6: 0.8131 - auc_6: 0.8451
Epoch 24/30
58/58 [==============================] - 18s 311ms/step - loss: 0.6603 - accuracy: 0.8050 - precision_6: 0.7706 - recall_6: 0.8318 - auc_6: 0.8687
Epoch 25/30
58/58 [==============================] - 18s 311ms/step - loss: 0.6080 - accuracy: 0.8313 - precision_6: 0.8044 - recall_6: 0.8458 - auc_6: 0.8971
Epoch 26/30
58/58 [==============================] - 18s 312ms/step - loss: 0.5955 - accuracy: 0.8423 - precision_6: 0.8170 - recall_6: 0.8551 - auc_6: 0.9008
Epoch 27/30
58/58 [==============================] - 18s 314ms/step - loss: 0.6107 - accuracy: 0.8456 - precision_6: 0.8154 - recall_6: 0.8668 - auc_6: 0.8874
Epoch 28/30
58/58 [==============================] - 18s 3

58/58 [==============================] - 18s 308ms/step - loss: 0.9595 - accuracy: 0.8149 - precision_8: 0.7534 - recall_8: 0.8995 - auc_8: 0.8548
Epoch 14/30
58/58 [==============================] - 18s 315ms/step - loss: 1.0218 - accuracy: 0.7492 - precision_8: 0.6940 - recall_8: 0.8318 - auc_8: 0.8215
Epoch 15/30
58/58 [==============================] - 18s 311ms/step - loss: 1.0227 - accuracy: 0.8039 - precision_8: 0.7632 - recall_8: 0.8435 - auc_8: 0.8244
Epoch 16/30
58/58 [==============================] - 18s 311ms/step - loss: 0.9144 - accuracy: 0.8105 - precision_8: 0.7766 - recall_8: 0.8364 - auc_8: 0.8498
Epoch 17/30
58/58 [==============================] - 18s 318ms/step - loss: 0.8705 - accuracy: 0.8028 - precision_8: 0.7551 - recall_8: 0.8575 - auc_8: 0.8584
Epoch 18/30
58/58 [==============================] - 18s 312ms/step - loss: 0.8188 - accuracy: 0.8302 - precision_8: 0.7725 - recall_8: 0.9042 - auc_8: 0.8751
Epoch 19/30
58/58 [==============================] - 18s 3